In [11]:
import numpy as np
import pandas as pd
import argparse
import json
from datetime import datetime
from copy import deepcopy
import random
import os
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import patches as mpatches
plt.style.use('seaborn')
import seaborn as sns
import time
import sys
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import warnings
warnings.filterwarnings('ignore');

# display multiple outputs within a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all";

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
%load_ext autoreload
%autoreload 2
# from params import MEGLpParams
from utils.stat_utils import metric_analysis,slice_metric_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
DATA_ROOT=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\data\MEG'
clinical_file=os.path.join(DATA_ROOT,'MEG.clinical.csv')
clinical_df = pd.read_csv(clinical_file)

In [ ]:
# ## let's just start with one directory..
# embedding_dir=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\ds\pats_CogTr1\L3\HGCN_full_findc_id_dp'
# emb_stat_paths=[os.path.join(embedding_dir,f) for f in os.listdir(embedding_dir) if 'embedding_stats_' in f]
# if emb_stat_paths:
#     if len(emb_stat_paths)>1:
#         print('should only be one embedding path but so what')
#     print(os.path.exists(emb_stat_paths[0]))
#     print(emb_stat_paths[0])
#     stat_df=pd.read_csv(emb_stat_paths[0])
#     output_dir=os.path.join(embedding_dir,"outputs","stat_analysis")
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
#     plot_save_path=os.path.join(output_dir,'anova_plot')
#     df_save_path=os.path.join(output_dir,'anova_df')
#     title='RM ANOVA analysis'
    
#     slice_metric_df(stat_df,label_col='diagnosis',title=title,plot_save_path=plot_save_path,df_save_path=df_save_path)

In [6]:
def run_anova(emb_stats,output_dir,title='',label_col='diagnosis',cond_dict={}):
    """
    emb_stats either path or pandas dataframe
    """
#     embedding_dir=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\ds\pats_CogTr1\L3\HGCN_full_findc_id_dp'
#     emb_stat_paths=[os.path.join(embedding_dir,f) for f in os.listdir(embedding_dir) if 'embedding_stats_' in f]
#     print(emb_stats,'STAT PATH')
#     if
    if type(emb_stats)==str:
        stat_df=pd.read_csv(emb_stats)
    else:
        stat_df=deepcopy(emb_stats)
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    plot_save_path=os.path.join(output_dir,'anova_plot'+title)
    df_save_path=os.path.join(output_dir,'anova_df'+title)
    if not title:
        title='RM ANOVA analysis'
    print(stat_df.shape,'STAT DICT BEFORE')
    stat_df_before=stat_df
    for condition,val in cond_dict.items():
        stat_df=stat_df[stat_df[condition]==val]
        title='{}_{}{}'.format(title,condition,val)
#         title=title+'_'+condition+'=='+val
#     print(cond_dict)
#     print(title)
    print(stat_df_before.shape,'BEFORE HAND')
    print(stat_df.shape,'STAT SHAPE AFTER COND')
    if not stat_df.shape[0]:
        print(stat_df_before,'WHAT HAPPENED TO EEVRYTHINGN')
    
#     assert False
#     return

    slice_metric_df(stat_df,label_col=label_col,title=title,plot_save_path=plot_save_path,df_save_path=df_save_path)

In [33]:
def run_anova_run_dir(run_dir,cond_dict={}):
    files=os.listdir(run_dir)
    outdir=os.path.join(run_dir,"outputs","stat_analysis")
    assert 'config.json' in files
    assert 'embeddings' in files
    embeddings_dir=os.path.join(run_dir,'embeddings')
    emb_stat_paths=[os.path.join(embeddings_dir,f) for f in os.listdir(embeddings_dir) if ('embedding_stats_' in f) and ('.csv' in f)]
#     assert emb_stat_paths,'change this later, should be able to handle things that dont exist'
    if len(emb_stat_paths)>1:
        print('should only be one embedding path but so what')
#     emb_stat_df=pd.read_csv(emb_stat_paths[0])
    emb_stat_df=pd.read_csv(os.path.join(run_dir,'scan_info_full.csv'))
    
    with open(os.path.join(run_dir,'config.json')) as f:
        config=json.load(f)
        
    if config['train_only']:
        title='All Embeddings (train_only)'
        run_anova(emb_stat_df,output_dir=outdir,title=title,label_col='diagnosis',cond_dict=cond_dict)
    else:
        for group,members in config['idxs_dict'].items():
            member_set=set(members)
            emb_stat_df_sub=emb_stat_df[emb_stat_df['Scan Index'].isin(member_set)]
            title='{} Embeddings'.format(group)
            print(group)
            print(len(member_set),emb_stat_df_sub.shape,'member shapes')
            if len(member_set)<8:
                  print('too small to judge!')
                  continue
            run_anova(emb_stat_df_sub,output_dir=outdir,title=title,label_col='diagnosis',cond_dict=cond_dict)
    print(config)
    

In [36]:
# run_dir=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hgcn\study\meg\lp\alpha_plv_54c_dp\4'
# run_dir=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\all_pats\L3\HGCN_full_0.54c_plv_dp\e100_p10_lr0.021_vpct0.3_tpct0.2_stretch98_b8\0'
# run_dir=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\ds\all_pats\L3\HGCN_full_0.54c_id_dp\0'
run_dir=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\ds\all_pats\L3\HGCN_full_0.54c_plv_dp\e60_p6_lr0.021_vpct0_tpct0_stretch98_b8\6'
run_dir=r"C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\all_pats\L3\HGCN_full_findc_plv_dp\e100_p6_lr0.021_vpct0.3_tpct0.2_strchinp95_strchloss95_b8\0"
run_dir=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hgcn\study\meg\lp\pats_CogTr1\L3\HGCN_full_0.54c_plv_dp\e100_p10_lr0.021_vpct0_tpct0_strchinp95_strchloss95_b8\6'
run_dir=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hgcn\study\meg\lp\pats_CogTr1\L3\HGCN_full_0.54c_plv_dp\e100_p10_lr0.021_vpct0_tpct0_strchinp95_strchloss95_b8\14'
run_dir=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\test\all_pats\L3\HGCN_full_findc_id_dp\e100_p6_lr0.021_vpct0.3_tpct0.2_strchinp95_strchloss95_b8_lmse\1'
run_dir=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\test\pats_CogTr1\L3\HGCN_full_findc_id_dp\e100_p6_lr0.021_val_excl_group_strchinp95_strchloss95_b8_lmse_pnuni\0'
cond_dict={'CogTr':1}
run_anova_run_dir(run_dir,cond_dict)

KeyError: 'Scan Index'

In [9]:
def run_anova_all_in_subtree(root,insist_substrs=[],cond_dict={}):
    for (subroot,dirs,files) in os.walk(root, topdown=True):
        
        if not insist_substrs:
            keep=True
        else:
            keep=False
            for s in insist_substrs:
                if s in subroot:
                    keep=True
        if not keep:
            continue
        print()
        emb_stat_paths=[os.path.join(subroot,f) for f in files if ('embedding_stats_' in f) and ('.csv' in f)]    
        if not emb_stat_paths:
            continue
#         print(subroot,'SUBROOT')
#         continue
        if len(emb_stat_paths)>1:
            print('should only be one embedding path but so what')
        for emb_path in emb_stat_paths:
            if 'average' in emb_path:
                file=os.path.basename(emb_path)
#                 print(emb_path)
                splits=file.split('_')
                print(splits)
                avg_indx=splits.index('average')
                title= 'Average of {} embeddings'.format(splits[avg_indx+1])
            else:
                title=''
            print(emb_path,'embeds')
            print(title,'title')
            outdir=os.path.join(subroot,"outputs","stat_analysis")
            run_anova(emb_path,output_dir=outdir,title=title,label_col='diagnosis',cond_dict=cond_dict)


In [40]:
cond_dict={'CogTr':1}

# run_anova_all_in_subtree(r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\ds\all_pats\L3\HGCN_full_0.54c_plv_dp',cond_dict=cond_dict)
# run_anova_all_in_subtree(r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\ds\all_pats\L3',['e20'],cond_dict)
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\ds\pats_CogTr1\L3\HGCN_full_0.54c_plv_dp\e5_p6_lr0.021_vpct0_tpct0_stretch98_b8'
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\ds\pats_CogTr1\L3\HGCN_full_0.54c_id_dp'
# root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\ds\pats_CogTr1\L3'
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hgcn\logs\meg\lp\2023_2_27'
root=r"C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hgcn\study\meg\lp\all_pats\L3\HGCN_full_0.54c_plv_dp\e100_p10_lr0.021_vpct0_tpct0_strchinp95_strchloss95_b8"
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\ds\all_pats\L3\HGCN_full_0.54c_plv_dp\e30_p6_lr0.021_vpct0_tpct0_strchinp95_strchloss95_b8'
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hgcn\study\meg\lp\pats_CogTr1\L3\HGCN_full_0.54c_plv_dp\e100_p10_lr0.021_vpct0_tpct0_strchinp95_strchloss95_b8'
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hgcn\logs\meg\lp\2023_3_6'
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\ds\all_pats\L3\HGCN_full_0.54c_plv_dp\e100_p6_lr0.021_vpct0_tpct0_strchinp95_strchloss95_b8'
# root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hgcn\study\meg\lp\pats_CogTr1\L3\HGCN_full_0.54c_plv_dp\e100_p10_lr0.021_vpct0_tpct0_strchinp95_strchloss95_b8'
# root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\all_pats\L3\HGCN_full_findc_plv_dp\e80_p6_lr0.021_vpct0.2_tpct0.01_strchinp95_strchloss95_b8'
# root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\all_pats\L3\HGCN_full_0.54c_plv_dp\e80_p6_lr0.021_vpct0.2_tpct0.01_strchinp95_strchloss95_b8'
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\pats_CogTr1\L3\HGCN_full_findc_plv_dp\e80_p6_lr0.021_val_excl_group_strchinp95_strchloss95_b8'
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hgcn\study\meg\lp\pats_CogTr1\L3\HGCN_full_0.54c_plv_dp\e100_p10_lr0.021_val_excl_group_strchinp95_strchloss95_b8'
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\pats_CogTr1\L3\HGCN_full_0.54c_plvid_dp\e100_p6_lr0.021_val_excl_group_strchinp95_strchloss95_b8'
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\pats_CogTr1\L3\HGCN_full_0.findc_plvid_dp\e100_p6_lr0.021_val_excl_group_strchinp95_strchloss95_b8'
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\test\all_pats'
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\test\pats_CogTr1\L3\HGCN_full_findc_plvid_dp'
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\test\pats_CogTr1\L3\HGCN_full_findc_id_dp\e100_p6_lr0.021_val_excl_group_strchinp95_strchloss95_b8_lmse_pnuni'
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\test\pats_CogTr1\L3\HGCN_full_findc_plvid_dp\e100_p6_lr0.021_val_excl_group_strchinp95_strchloss95_b8_lmse_pnuni'
run_anova_all_in_subtree(root,cond_dict=cond_dict)

In [47]:
# root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\all_pats\L3\HGCN_full_0.54c_plv_dp\e100_p10_lr0.021_vpct0.3_tpct0.2_stretch98_b8'
# root='C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\all_pats\L3'
cond_dict={'CogTr':1}
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\ds\all_pats\L3\HGCN_full_0.54c_plv_dp\e60_p6_lr0.021_vpct0_tpct0_stretch98_b8\6'
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\ds\all_pats\L3\HGCN_full_0.54c_plv_dp\e50_p6_lr0.021_vpct0_tpct0_strchinp95_strchloss95_b8'
root=r"C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hgcn\study\meg\lp\pats_CogTr1\L3\HGCN_full_0.54c_plv_dp\e100_p10_lr0.021_vpct0_tpct0_strchinp95_strchloss95_b8"
# root=r"C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hgcn\study\meg\lp\all_pats\L3\HGCN_full_0.54c_plv_dp\e100_p10_lr0.021_vpct0_tpct0_strchinp95_strchloss95_b8"
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\pats_CogTr1\L3\HGCN_full_0.54c_plv_dp\e80_p6_lr0.021_val_excl_group_strchinp92_strchloss95_b8'
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\pats_CogTr1\L3\HGCN_full_findc_plv_dp\e80_p6_lr0.021_val_excl_group_strchinp97_strchloss95_b8'
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\pats_CogTr1\L3\HGCN_full_0.54c_plvid_dp\e100_p6_lr0.021_val_excl_group_strchinp95_strchloss95_b8'
# root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\pats_CogTr1\L3\HGCN_full_findc_plvid_dp\e100_p6_lr0.021_val_excl_group_strchinp95_strchloss95_b8'
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\test\pats_CogTr1\L3\HGCN_full_findc_id_dp\e100_p7_lr0.021_val_excl_group_strchinp95_strchloss95_b8_lmse_pneq'
root=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\test\pats_CogTr1\L2\HGCN_full_findc_id_dp\e100_p7_lr0.021_val_excl_group_strchinp95_strchloss95_b8_lmse_pneq'
root=r' C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\test\all_pats\L4\HGCN_full_findc_id_dp\e100_p2fr5_lr0.01_vpct0.3_tpct0.2_strchinp95_strchloss95_bn_lmse_pneq'
run_anova_all_in_subtree(root,cond_dict=cond_dict)